In [27]:
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        paths = os.path.join(dirname, filename)

In [28]:

path = "https://raw.githubusercontent.com/ResidenciaTICBrisa/06_AcompanhamentoEnsinoMedio/develop/src/ML/student_data.csv"

df = pd.read_csv(path)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [29]:
display(df)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [30]:
df_delColunas = df.drop(["health","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","guardian","traveltime","schoolsup","paid","famsup","activities","nursery","romantic","famrel","freetime","goout","Dalc","Walc"], axis=1)
display(df_delColunas)

,school,sex,age,studytime,failures,higher,internet,absences,G1,G2,G3
0,GP,F,18,2,0,yes,no,6,5,6,6
1,GP,F,17,2,0,yes,yes,4,5,5,6
2,GP,F,15,2,3,yes,yes,10,7,8,10
3,GP,F,15,3,0,yes,yes,2,15,14,15
4,GP,F,16,2,0,yes,no,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,2,2,yes,no,11,9,9,9
391,MS,M,17,1,0,yes,yes,3,14,16,16
392,MS,M,21,1,3,yes,no,3,10,8,7
393,MS,M,18,1,0,yes,yes,0,11,12,10


In [31]:
df_delColunas['Approved'] = df_delColunas.apply(lambda row: 1 if (((row['G1'] + row['G2'] + row['G3']) / 3 > 10) and row['absences'] <= 10) else 0, axis=1)

In [32]:
df_delColunas['absences'].unique()

array([ 6,  4, 10,  2,  0, 16, 14,  7,  8, 25, 12, 54, 18, 26, 20, 56, 24,
       28,  5, 13, 15, 22,  3, 21,  1, 75, 30, 19,  9, 11, 38, 40, 23, 17])

# Previsão Individual de G1, G2, G3 e Approved
## Regressão Linear

In [33]:
# Separate the predictor columns (X) and the target variables (y)
X_G1 = df_delColunas.drop(['G1', 'G2', 'G3', 'Approved'], axis=1)
X_G2 = df_delColunas.drop(['G2', 'G3', 'Approved'], axis=1)
X_G3 = df_delColunas.drop(['G3', 'Approved'], axis=1)
X_Approved = df_delColunas.drop('Approved', axis=1)

y_G1 = df_delColunas['G1']
y_G2 = df_delColunas['G2']
y_G3 = df_delColunas['G3']
y_Approved = df_delColunas['Approved']

# Perform one-hot encoding on the categorical features
X_G1_encoded = pd.get_dummies(X_G1)
X_G2_encoded = pd.get_dummies(X_G2)
X_G3_encoded = pd.get_dummies(X_G3)
X_Approved_encoded = pd.get_dummies(X_Approved)

# Split the data into training and testing sets
X_G1_train, X_G1_test, y_G1_train, y_G1_test = train_test_split(X_G1_encoded, y_G1, test_size=0.2, random_state=42)
X_G2_train, X_G2_test, y_G2_train, y_G2_test = train_test_split(X_G2_encoded, y_G2, test_size=0.2, random_state=42)
X_G3_train, X_G3_test, y_G3_train, y_G3_test = train_test_split(X_G3_encoded, y_G3, test_size=0.2, random_state=42)
X_Approved_train, X_Approved_test, y_Approved_train, y_Approved_test = train_test_split(X_Approved_encoded, y_Approved, test_size=0.2, random_state=42)

In [34]:
# Create and train the linear regression model for G1
regressor_G1 = LinearRegression()
regressor_G1.fit(X_G1_train, y_G1_train)

# Create and train the linear regression model for G2
regressor_G2 = LinearRegression()
regressor_G2.fit(X_G2_train, y_G2_train)

# Create and train the linear regression model for G3
regressor_G3 = LinearRegression()
regressor_G3.fit(X_G3_train, y_G3_train)

# Create and train the linear regression model for Approved
regressor_Approved = LinearRegression()
regressor_Approved.fit(X_Approved_train, y_Approved_train)

LinearRegression()

In [35]:
# Make predictions for G1, G2, and G3
y_G1_pred = regressor_G1.predict(X_G1_test)
y_G2_pred = regressor_G2.predict(X_G2_test)
y_G3_pred = regressor_G3.predict(X_G3_test)
y_Approved_pred = regressor_Approved.predict(X_Approved_test)

# Evaluate the linear regression model using Mean Absolute Error (MAE)
mae_G1 = mean_absolute_error(y_G1_test, y_G1_pred)
mae_G2 = mean_absolute_error(y_G2_test, y_G2_pred)
mae_G3 = mean_absolute_error(y_G3_test, y_G3_pred)
mae_Approved = mean_absolute_error(y_Approved_test, y_Approved_pred)

# # Print the predictions and the actual values side by side for G1
# print("Prediction for G1")
# for pred, real in zip(y_G1_pred, y_G1_test):
#     print(f"Predicted: {pred:.2f} - Actual: {real:.2f} => Diff: {(pred-real):.2f}")

# # Print the predictions and the actual values side by side for G2
# print("\nPrediction for G2")
# for pred, real in zip(y_G2_pred, y_G2_test):
#     print(f"Predicted: {pred:.2f} - Actual: {real:.2f} => Diff: {(pred-real):.2f}")

# # Print the predictions and the actual values side by side for G3
# print("\nPrediction for G3")
# for pred, real in zip(y_G3_pred, y_G3_test):
#     print(f"Predicted: {pred:.2f} - Actual: {real:.2f} => Diff: {(pred-real):.2f}")
    
# # Print the predictions and the actual values side by side for Approved
# print("\nPrediction for Approved")
# for pred, real in zip(y_Approved_pred, y_Approved_test):
#     print(f"Predicted: {pred:.2f} - Actual: {real:.2f} => Diff: {(pred-real):.2f}")

# Print the Mean Absolute Error (MAE) for G1, G2, and G3
print("Mean Absolute Error (G1):", mae_G1)
print("Mean Absolute Error (G2):", mae_G2)
print("Mean Absolute Error (G3):", mae_G3)
print("Mean Absolute Error (Approved):", mae_Approved)

Mean Absolute Error (G1): 2.9942021883275505
Mean Absolute Error (G2): 1.1598800728672158
Mean Absolute Error (G3): 1.346370714289203
Mean Absolute Error (Approved): 0.23050038977116102


## Redes Neurais

In [36]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Load the DataFrame
df_delColunas = pd.read_csv(path)

df_delColunas['Approved'] = df_delColunas.apply(lambda row: 1 if (((row['G1'] + row['G2'] + row['G3']) / 3 > 10) and row['absences'] <= 10) else 0, axis=1)

# Separate the predictor columns (X) and the target variables (y)
X_G1 = df_delColunas.drop(['G1', 'G2', 'G3', 'Approved'], axis=1)
X_G2 = df_delColunas.drop(['G2', 'G3', 'Approved'], axis=1)
X_G3 = df_delColunas.drop(['G3', 'Approved'], axis=1)
X_Approved = df_delColunas.drop('Approved', axis=1)

y_G1 = df_delColunas['G1']
y_G2 = df_delColunas['G2']
y_G3 = df_delColunas['G3']
y_Approved = df_delColunas['Approved']

# Perform one-hot encoding on the categorical features
X_G1_encoded = pd.get_dummies(X_G1)
X_G2_encoded = pd.get_dummies(X_G2)
X_G3_encoded = pd.get_dummies(X_G3)
X_Approved_encoded = pd.get_dummies(X_Approved)

# Split the data into training and testing sets
X_G1_train, X_G1_test, y_G1_train, y_G1_test = train_test_split(X_G1_encoded, y_G1, test_size=0.2, random_state=42)
X_G2_train, X_G2_test, y_G2_train, y_G2_test = train_test_split(X_G2_encoded, y_G2, test_size=0.2, random_state=42)
X_G3_train, X_G3_test, y_G3_train, y_G3_test = train_test_split(X_G3_encoded, y_G3, test_size=0.2, random_state=42)
X_Approved_train, X_Approved_test, y_Approved_train, y_Approved_test = train_test_split(X_Approved_encoded, y_Approved, test_size=0.2, random_state=42)

In [40]:
# Create and train the neural network model for G1
model_G1 = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(X_G1_train.shape[1],)),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1)
])

model_G1.compile(optimizer='adam', loss='mean_squared_error')

model_G1.fit(X_G1_train, y_G1_train, epochs=100, batch_size=32, verbose=0)

# Create and train the neural network model for G2
model_G2 = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(X_G2_train.shape[1],)),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1)
])

model_G2.compile(optimizer='adam', loss='mean_squared_error')

model_G2.fit(X_G2_train, y_G2_train, epochs=100, batch_size=32, verbose=0)

# Create and train the neural network model for G3
model_G3 = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(X_G3_train.shape[1],)),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1)
])

model_G3.compile(optimizer='adam', loss='mean_squared_error')

model_G3.fit(X_G3_train, y_G3_train,epochs=100, batch_size=32, verbose=0)

# Create and train the neural network model for Approved
model_Approved = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(X_Approved_train.shape[1],)),
    tf.keras.layers.Dense(15, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_Approved.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_Approved.fit(X_Approved_train, y_Approved_train, epochs=100, batch_size=32, verbose=0)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [38]:
# Make predictions for G1, G2, G3, and Approved
y_G1_pred = model_G1.predict(X_G1_test).flatten()
y_G2_pred = model_G2.predict(X_G2_test).flatten()
y_G3_pred = model_G3.predict(X_G3_test).flatten()
y_Approved_pred = model_Approved.predict(X_Approved_test).flatten()

# Calculate and print the Mean Absolute Error (MAE) for G1, G2, G3, and Approved
mae_G1 = mean_absolute_error(y_G1_test, y_G1_pred)
mae_G2 = mean_absolute_error(y_G2_test, y_G2_pred)
mae_G3 = mean_absolute_error(y_G3_test, y_G3_pred)
mae_Approved = mean_absolute_error(y_Approved_test, y_Approved_pred)

print("Mean Absolute Error (G1):", mae_G1)
print("Mean Absolute Error (G2):", mae_G2)
print("Mean Absolute Error (G3):", mae_G3)
print("Mean Absolute Error (Approved):", mae_Approved)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

# Previsão das Notas G1, G2, G3 e Approved em cima das próprias previsões

In [39]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Load the DataFrame
df_delColunas = pd.read_csv(path)

df_delColunas['Approved'] = df_delColunas.apply(lambda row: 1 if (((row['G1'] + row['G2'] + row['G3']) / 3 > 10) and row['absences'] <= 10) else 0, axis=1)

# Separate the predictor columns (X) and the target variables (y)
X_G1 = df_delColunas.drop(['G1', 'G2', 'G3', 'Approved'], axis=1)
X_G2 = df_delColunas.drop(['G2', 'G3', 'Approved'], axis=1)
X_G3 = df_delColunas.drop(['G3', 'Approved'], axis=1)
X_Approved = df_delColunas.drop('Approved', axis=1)

y_G1 = df_delColunas['G1']
y_G2 = df_delColunas['G2']
y_G3 = df_delColunas['G3']
y_Approved = df_delColunas['Approved']

# Perform one-hot encoding on the categorical features
X_G1_encoded = pd.get_dummies(X_G1)
X_G2_encoded = pd.get_dummies(X_G2)
X_G3_encoded = pd.get_dummies(X_G3)
X_Approved_encoded = pd.get_dummies(X_Approved)

# Split the data into training and testing sets
X_G1_train, X_G1_test, y_G1_train, y_G1_test = train_test_split(X_G1_encoded, y_G1, test_size=0.2, random_state=42)
X_G2_train, X_G2_test, y_G2_train, y_G2_test = train_test_split(X_G2_encoded, y_G2, test_size=0.2, random_state=42)
X_G3_train, X_G3_test, y_G3_train, y_G3_test = train_test_split(X_G3_encoded, y_G3, test_size=0.2, random_state=42)
X_Approved_train, X_Approved_test, y_Approved_train, y_Approved_test = train_test_split(X_Approved_encoded, y_Approved, test_size=0.2, random_state=42)

# Create and train the neural network model for G1
model_G1 = tf.keras.Sequential([
    tf.keras.layers.Dense(56, activation='relu', input_shape=(X_G1_train.shape[1],)),
    tf.keras.layers.Dense(42, activation='relu'),
    tf.keras.layers.Dense(36, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1)
])
model_G1.compile(optimizer='adam', loss='mean_squared_error')
model_G1.fit(X_G1_train, y_G1_train, epochs=200, batch_size=28, verbose=0)
# Predict G1 for the test set
y_G1_pred = model_G1.predict(X_G1_test)

# Create a new DataFrame for X_G2 with the predicted G1 values
X_G2_pred = X_G2_test.copy()
X_G2_pred['G1'] = y_G1_pred
# Create and train the neural network model for G2
model_G2 = tf.keras.Sequential([
    tf.keras.layers.Dense(57, activation='relu', input_shape=(X_G2_pred.shape[1],)),
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.Dense(1)
])
model_G2.compile(optimizer='adam', loss='mean_squared_error')
model_G2.fit(X_G2_pred, y_G2_test, epochs=300, batch_size=32, verbose=0)
# Predict G2 for the test set
y_G2_pred = model_G2.predict(X_G2_test)

# Create a new DataFrame for X_G3 with the predicted G1 and G2 values
X_G3_pred = X_G3_test.copy()
X_G3_pred['G1'] = y_G1_pred
X_G3_pred['G2'] = y_G2_pred
# Create and train the neural network model for G3
model_G3 = tf.keras.Sequential([
    tf.keras.layers.Dense(58, activation='relu', input_shape=(X_G3_pred.shape[1],)),
    tf.keras.layers.Dense(29, activation='relu'),
    tf.keras.layers.Dense(1)
])
model_G3.compile(optimizer='adam', loss='mean_squared_error')
model_G3.fit(X_G3_pred, y_G3_test, epochs=300, batch_size=32, verbose=0)
# Predict G3 for the test set
y_G3_pred = model_G3.predict(X_G3_test)

# Create a new DataFrame for X_Approved with the predicted G1, G2, and G3 values
X_Approved_pred = X_Approved_test.copy()
X_Approved_pred['G1'] = y_G1_pred
X_Approved_pred['G2'] = y_G2_pred
X_Approved_pred['G3'] = y_G3_pred
# Create and train the neural network model for Approved
model_Approved = tf.keras.Sequential([
    tf.keras.layers.Dense(59, activation='relu', input_shape=(X_Approved_pred.shape[1],)),
    tf.keras.layers.Dense(34, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_Approved.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_Approved.fit(X_Approved_pred, y_Approved_test, epochs=35, batch_size=32, verbose=0)

# Make predictions for G1, G2, G3, and Approved
y_G1_pred = model_G1.predict(X_G1_test).flatten()
y_G2_pred = model_G2.predict(X_G2_test).flatten()
y_G3_pred = model_G3.predict(X_G3_test).flatten()
y_Approved_pred = model_Approved.predict(X_Approved_test).flatten()

# Calculate and print the Mean Absolute Error (MAE) for G1, G2, G3, and Approved
mae_G1 = mean_absolute_error(y_G1_test, y_G1_pred)
mae_G2 = mean_absolute_error(y_G2_test, y_G2_pred)
mae_G3 = mean_absolute_error(y_G3_test, y_G3_pred)
mae_Approved = mean_absolute_error(y_Approved_test, y_Approved_pred)

print("Mean Absolute Error (G1):", mae_G1)
print("Mean Absolute Error (G2):", mae_G2)
print("Mean Absolute Error (G3):", mae_G3)
print("Mean Absolute Error (Approved):", mae_Approved)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

# Testando previsões em estudante aleatório

In [ ]:
import numpy as np

# Escolha o índice do estudante aleatório do conjunto de teste
random_student_index = 45

# Obtenha a linha correspondente aos dados desse estudante no conjunto de teste
X_G1_test_student = np.array(X_G1_test.iloc[random_student_index]).reshape(1, -1)
X_G2_test_student = np.array(X_G2_test.iloc[random_student_index]).reshape(1, -1)
X_G3_test_student = np.array(X_G3_test.iloc[random_student_index]).reshape(1, -1)
X_Approved_test_student = np.array(X_Approved_test.iloc[random_student_index]).reshape(1, -1)

# Faça a previsão para G1, G2, G3 e Approved para esse estudante
y_G1_pred_student = model_G1.predict(X_G1_test_student)[0][0]
y_G2_pred_student = model_G2.predict(X_G2_test_student)[0][0]
y_G3_pred_student = model_G3.predict(X_G3_test_student)[0][0]
y_Approved_pred_student = model_Approved.predict(X_Approved_test_student)[0][0]

# Obtenha os valores reais para G1, G2, G3 e Approved para esse estudante
y_G1_actual_student = y_G1_test.iloc[random_student_index]
y_G2_actual_student = y_G2_test.iloc[random_student_index]
y_G3_actual_student = y_G3_test.iloc[random_student_index]
y_Approved_actual_student = y_Approved_test.iloc[random_student_index]

# Imprima as previsões e os valores reais para G1, G2, G3 e Approved do estudante selecionado
print("\nG1")
print(f"Predicted: {y_G1_pred_student:.2f} - Actual: {y_G1_actual_student:.2f}")

print("\nG2")
print(f"Predicted: {y_G2_pred_student:.2f} - Actual: {y_G2_actual_student:.2f}")

print("\nG3")
print(f"Predicted: {y_G3_pred_student:.2f} - Actual: {y_G3_actual_student:.2f}")

print("\nApproved")
print(f"Predicted: {y_Approved_pred_student:.0f} - Actual: {y_Approved_actual_student}")

1/1 [==============================] - 0s 26ms/step

G1
Predicted: 8.34 - Actual: 11.00

G2
Predicted: 11.52 - Actual: 10.00

G3
Predicted: 9.47 - Actual: 10.00

Approved
Predicted: 1 - Actual: 1
